In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import  LogisticRegression
from sklearn.metrics import accuracy_score
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
#read data
df = pd.read_csv('/content/drive/MyDrive/MlLab/spam.csv', encoding='ISO-8859-1')
#remove last 3 null value colum
df = df.iloc[:, :-3]
df.columns = ['category', 'message']
#remove null value
df = df.dropna()
df.head()
# df['category'].value_counts()

#label encoding spam as 0 and ham is 1
# df['category']=df['category'].map({'spam': 0, 'ham': 1})

,category,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('category').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('message').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['message'].value_counts()
    for x_label, grp in _df_2.groupby('category')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('category')
_ = plt.ylabel('message')

In [ ]:
import nltk
from nltk import RegexpTokenizer

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [ ]:
tokenizer = RegexpTokenizer('\w+')
stop_words_set = set(stopwords.words('english'))
porter_stemmer = PorterStemmer()

In [ ]:
def preprocess(raw_text):
    raw_text = raw_text.lower()
    tokens = tokenizer.tokenize(raw_text) # breaking into small words
    # print(tokens)
    removed_stopwords = [word for word in tokens if word not in stop_words_set]
    # print(removed_stopwords)
    stemmed_words = [porter_stemmer.stem(token) for token in removed_stopwords]
    # print(stemmed_words)
    processed_text = ' '.join(stemmed_words)
    return processed_text

In [ ]:
preprocess("Hi! I am XYZ. Don't talked too much.")

'hi xyz talk much'

In [ ]:
df['message'] = df['message'].map(preprocess)

df.head()

,category,message
0,ham,go jurong point crazi avail bugi n great world...
1,ham,ok lar joke wif u oni
2,spam,free entri 2 wkli comp win fa cup final tkt 21...
3,ham,u dun say earli hor u c alreadi say
4,ham,nah think goe usf live around though


In [ ]:
X = df['message']
y = df['category']
X.shape, y.shape


((5572,), (5572,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
spam_words_probability = dict()
ham_words_probability = dict ()

def fit(X_train, y_train):
  spam_dict = dict()
  ham_dict = dict()

  spam_words_count = 0
  ham_words_count = 0

  # assign count 1 to every training words in the dataset
  for temp in X_train:
    tokens = tokenizer.tokenize(temp)
    for words in tokens:
      spam_dict[words]=1
      ham_dict[words]=1

  # count spam words and ham words
  for (sentence,types) in zip(X_train,y_train):
    tokens = tokenizer.tokenize(sentence)

    for words in tokens:
      if types == 'spam':
        spam_dict[words]= spam_dict[words] +1

      else:
        ham_dict[words] = ham_dict[words] +1

  # count total words in ham and spam
  for temp in ham_dict:
    ham_words_count = ham_words_count + ham_dict [temp]
  for temp in spam_dict:
    spam_words_count = spam_words_count + spam_dict [temp]

  # count the probabilities
  for temp in ham_dict:
    ham_words_probability[temp] = ham_dict[temp]/ham_words_count
  for temp in spam_dict:
    spam_words_probability[temp] = spam_dict[temp]/spam_words_count

fit(X_train,y_train)

In [ ]:
spam_count = 0
ham_count =0

for type in y_train:
  if type == 'spam':
    spam_count = spam_count + 1
  else :
    ham_count = ham_count + 1


def predict(message):
  text=preprocess(message)
  tokens = tokenizer.tokenize(text)

  spam_prob= spam_count/(spam_count + ham_count)
  ham_prob = ham_count/(spam_count+ham_count)
  for words in tokens:
    if words in spam_words_probability:
      spam_prob = spam_prob * spam_words_probability[words]
      ham_prob = ham_prob * ham_words_probability[words]

  if ham_prob > spam_prob:
    return 'ham'
  else :
    return 'spam'

predict('Yes...I think goe usf live around though	')

'ham'

In [ ]:
def evaluate(X_test, y_test):

  correct = 0
  for (msg,type) in zip (X_test, y_test):
    check = predict(msg)
    if check == type:
      correct = correct + 1
  print("Accuract :")
  print(correct)
  print("Message :")
  print(len(y_test))
  correct = (correct/len(y_test)) * 100
  print("Accuracy :")
  print(correct)

evaluate(X_test, y_test)

Accuract :
552
Message :
558
Accuracy :
98.9247311827957
